<a href="https://colab.research.google.com/github/slipaway/melonplaylist/blob/master/tag2vec%20model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 전처리 하기 위한 준비

In [1]:
#형태소 분석기
!git clone https://github.com/kakao/khaiii.git
!pip install cmake
!mkdir build
!cd build && cmake /content/khaiii
!cd /content/build/ && make all
!cd /content/build/ && make resource
!cd /content/build && make install
!cd /content/build && make package_python
!pip install /content/build/package_python

Cloning into 'khaiii'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 957 (delta 25), reused 49 (delta 19), pack-reused 877
Receiving objects: 100% (957/957), 33.03 MiB | 9.79 MiB/s, done.
Resolving deltas: 100% (383/383), done.
-- [hunter] Initializing Hunter workspace (70287b1ffa810ee4e952052a9adff9b4856d0d54)
-- [hunter]   https://github.com/ruslo/hunter/archive/v0.23.34.tar.gz
-- [hunter]   -> /root/.hunter/_Base/Download/Hunter/0.23.34/70287b1
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
--

In [2]:
#구글 계정 마운트
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from datetime import timedelta, datetime
import glob
import os
import re
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import re
from collections import Counter
from typing import *
from khaiii import KhaiiiApi

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
folder = '/content/gdrive/My Drive/멜론 플레이리스트' #working directory
filelist = os.chdir(folder)

In [5]:
train=pd.read_json('train.json')
test = pd.read_json('test.json')

In [ ]:
!git clone https://github.com/slipaway/melonplaylist.git

# 0. 함수 정의

In [6]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

def uppertolower(string):
  if isEnglish(string) == True:
    string = string.lower() #태그나 제목이 영어로 들어간 경우, 모두 소문자로 바꿔주기

In [7]:
def re_sub(series: pd.Series) -> pd.Series:
  series = series.str.replace(pat=r'[ㄱ-ㅎ]', repl=r'', regex=True) #ㅋ 제거용
  series = series.str.replace(pat=r'[^\w\s]', repl=r'', regex=True) #특수문자 제거용
  series = series.str.replace(pat=r'[ ]{2,}', repl=r' ', regex=True) #공백 제거
  series = series.str.replace(pat=r'[\u3000]+', repl=r'', regex=True) #u3000제거
  return series

def flatten(list_of_list : List) -> List:
  flatten = [j for i in list_of_list for j in i]
  return flatten

def get_token(title: str, tokenizer)-> List[Tuple]:

  if len(title)==0 or title== ' ' : #제목이 공백인 경우 tokenizer 에러 발생
    return []
  
  result = tokenizer.analyze(title)
  result = [(morph.lex, morph.tag) for split in result for morph in split.morphs] # (형태소, 품사) 튜플의 리스트
  return result

def get_all_tags(df) -> List:
  tag_list = df['tags'].values.tolist()
  tag_list=flatten(tag_list)
  return tag_list

# 1. 태그 전처리

In [8]:
 train['tags'].head(10)

0                                                  [락]
1                                             [추억, 회상]
2                                            [까페, 잔잔한]
3    [연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...
4                                                 [댄스]
5    [운동, 드라이브, Pop, 트로피컬하우스, 힐링, 기분전환, 2017, 팝, 트렌...
6                          [짝사랑, 취향저격, 슬픔, 고백, 사랑, 이별]
7                                        [잔잔한, 추억, 회상]
8                          [일렉트로니카, 포크, 메탈, 락, 댄스, 인디]
9                   [록, Metal, 이일우, M에센셜, 메탈, Rock, 락]
Name: tags, dtype: object

In [9]:
#영어는 소문자로 모두 바꾸기
for i in range(len(train['tags'])):
  for j in range(len(train['tags'][i])):
    train['tags'][i][j]= train['tags'][i][j].lower()

In [10]:
tokenizer = KhaiiiApi()
all_tag = get_all_tags(train)
token_tag = [get_token(x, tokenizer) for x in all_tag] #태그를 형태소 분석

In [ ]:
token_tag[:40]

In [ ]:
token_itself=list(filter(lambda x: len(x)==1, token_tag)) # 태그 자체가 형태소여서 분리되지 않는 태그만 고름
token_itself=flatten(token_itself)
flatten_token=flatten(token_tag)

In [ ]:
print('%-23s'%'# of original tag is', f'{len(all_tag):8,}')
print('%-23s'%'# of morpheme itself is', f'{len(token_itself):8,}')
print('%-23s'%'# of total token is', f'{len(flatten_token):8,}')

# of original tag is     476,331
# of morpheme itself is  333,093
# of total token is      722,860


In [ ]:
#counting part of speech (pos)
pos=[x[1] for x in token_itself]
pos_count=Counter(pos)
popular_pos = pos_count.most_common()

In [ ]:
#tag 분류표
objects=[x[0] for x in popular_pos]
y_pos=np.arange(len(objects))
performance=[x[1] for x in popular_pos]

plt.barh(y_pos, performance, align='center', alpha=0.5)
plt.yticks(y_pos, objects)
plt.xlabel('Usage')
plt.title('Part of Speech - Tags')
plt.show()

# 2. 태그: word2vec 임베딩 with train dataset



In [11]:
from gensim.test.utils import get_tmpfile
from gensim.models.callbacks import CallbackAny2Vec
from gensim.models import Word2Vec

class EpochSaver(CallbackAny2Vec):
  def __init__(self, path_prefix):
    self.path_prefix = path_prefix
    self.epoch = 0

  def on_epoch_end(self, model):
    output_path = get_tmpfile('{}_epoch{}.model'.format(self.path_prefix, self.epoch))
    model.save(output_path)
    self.epoch += 1

class EpochLogger(CallbackAny2Vec):
     def __init__(self):
         self.epoch = 0

     def on_epoch_begin(self, model):
         print("Epoch #{} start".format(self.epoch))

     def on_epoch_end(self, model):
         print("Epoch #{} end".format(self.epoch))
         self.epoch += 1

In [ ]:
tag_list = [train['tags'][i] for i in range(len(train['tags']))]
epoch_logger = EpochLogger()
tag_embedding_model = Word2Vec(tag_list, window = 3, min_count=3, workers=4, iter=1000, sg=1, callbacks=[epoch_logger])

In [13]:
list(tag_embedding_model.wv.vocab.keys())

['락',
 '추억',
 '회상',
 '까페',
 '잔잔한',
 '연말',
 '눈오는날',
 '캐럴',
 '분위기',
 '따듯한',
 '크리스마스캐럴',
 '겨울노래',
 '크리스마스',
 '겨울왕국',
 '크리스마스송',
 '댄스',
 '운동',
 '드라이브',
 'pop',
 '트로피컬하우스',
 '힐링',
 '기분전환',
 '2017',
 '팝',
 '트렌드',
 '일렉',
 '짝사랑',
 '취향저격',
 '슬픔',
 '고백',
 '사랑',
 '이별',
 '일렉트로니카',
 '포크',
 '메탈',
 '인디',
 '록',
 'metal',
 'm에센셜',
 'rock',
 'kpop',
 '걸그룹댄스',
 '스트레스해소',
 '새해',
 '여행',
 '프로필음악',
 '카카오톡',
 '소원',
 '프로필',
 '소망',
 '다짐',
 '카톡',
 '듣고',
 '우울',
 '힘내',
 '힙합',
 '느낌있는',
 '밤',
 '새벽',
 'rnb',
 '감각적인',
 '국내',
 '그루브한',
 '가을',
 '재즈',
 '감성',
 '질리지않는',
 '나만알고싶은',
 '봄',
 '설렘',
 '비오는날',
 '스밍',
 '목록',
 '폐막식',
 '올림픽',
 '엑소',
 '조용히',
 '혼자',
 '새벽감성',
 '고민',
 '맥주한잔',
 '카페',
 'ost',
 '어쿠스틱',
 '편안한',
 '에너지',
 '듀엣',
 '달달',
 '피쳐링',
 '남녀',
 '신인',
 '국힙',
 '신나는',
 '존윌리엄스',
 '영화클래식',
 '겨울',
 '아침',
 '빅밴드',
 '상쾌한',
 '밝은',
 '스윙',
 '스윙재즈',
 '휴식',
 '여름',
 '매장음악',
 '헬스',
 '스포츠',
 '피트니스',
 '다이어트',
 '런닝',
 '필라테스',
 '산책',
 '요가',
 '발라드',
 '캐롤',
 '위로',
 '80년대',
 '90년대발라드',
 '올드송',
 '90년대',
 '옛날노래',
 '월드',
 '연주',
 '클럽',
 '스트레스',
 '주

예시.

In [14]:
tag_embedding_model.wv.most_similar('연말')

[('성탄절', 0.5692176222801208),
 ('연말연시', 0.5447402596473694),
 ('눈', 0.5366084575653076),
 ('송년회', 0.5153988599777222),
 ('메리크리스마스', 0.5093715786933899),
 ('크리스마스', 0.5024733543395996),
 ('해피뉴이어', 0.4958992600440979),
 ('겨울', 0.49529317021369934),
 ('캐럴', 0.492306113243103),
 ('새해', 0.48516005277633667)]

In [15]:
tag_embedding_model.wv.most_similar('조용')

[('조용한', 0.5051878690719604),
 ('진정', 0.4868573546409607),
 ('수면유도', 0.47777390480041504),
 ('공감하는', 0.4696665406227112),
 ('생각이', 0.46470576524734497),
 ('몰입', 0.4522252380847931),
 ('u2', 0.4449160099029541),
 ('생각을', 0.4394547939300537),
 ('가로등', 0.4385089576244354),
 ('아쿠스틱', 0.4308089017868042)]

In [16]:
tag_embedding_model.wv.most_similar(positive = ['감성','기분전환'])

[('휴식', 0.6638692617416382),
 ('드라이브', 0.5789991617202759),
 ('잔잔한', 0.5763711333274841),
 ('핫한노래', 0.49831366539001465),
 ('카페', 0.48796892166137695),
 ('당당하게', 0.4813615679740906),
 ('노래방차트', 0.4806048274040222),
 ('믿듣보컬', 0.4767851233482361),
 ('수빈', 0.46715590357780457),
 ('여행', 0.4572126567363739)]

# 3. 태그 예측하기

# Test: tag 있는 것들 중에서 tag 예측

In [17]:
#태그가 있는 플레이리스트 인덱스 뽑아내기
tag_yu_index = []
tag_no_index = []
for i in range(len(test['tags'])):
  if len(test['tags'][i]) != 0:
    tag_yu_index.append(i)
  else:
    tag_no_index.append(i)

In [18]:
test_tag_yu = test.loc[tag_yu_index,:]
test_tag_yu = test_tag_yu.reset_index()

In [19]:
test_tag_yu

,index,tags,id,plylst_title,songs,like_cnt,updt_date
0,1,"[나만의Best3, 인디아티스트들의추천음악]",7461,,"[196298, 269984, 267805, 175867, 529244, 63825...",0,2019-12-17 14:06:45.000
1,2,[드라이브],90348,,"[273433, 331003, 68432, 411659, 117793, 616860...",21,2015-05-23 10:44:48.000
2,3,[분위기],58617,,"[702227, 48152, 440008, 358488, 701041, 540721...",0,2019-03-14 09:47:34.000
3,5,[스트레스],88862,노래방 안에서,[],22,2016-01-22 20:55:12.000
4,7,[사랑],133975,봄바람과 듣고싶은 설레는 노래,[],0,2020-02-19 03:05:49.000
...,...,...,...,...,...,...,...
5417,10733,[별],77688,,"[93507, 324019, 221377, 656426, 26083, 661209,...",8,2018-08-10 20:14:19.000
5418,10735,[추억],137930,,"[323755, 397594, 445908, 570242, 221853, 20018...",16,2016-04-18 11:02:09.000
5419,10736,"[띵곡의, 우울, 분위기, 드라이브, 산책]",936,,"[105140, 582252, 199262, 422915, 547967, 48791...",1,2020-04-08 07:15:59.000
5420,10737,[기분전환],110589,,"[21976, 207746, 40025, 31635, 567462, 641799, ...",6,2016-06-29 00:57:21.000


In [21]:
test_tag_list = [list(filter(lambda x: x in list(tag_embedding_model.wv.vocab.keys()), test_tag_yu['tags'][i])) for i in range(len(test_tag_yu['tags']))]

In [ ]:
list_for_list = []
val_predicted_tag = []
for i in range(len(test_tag_list)):
  if len(test_tag_list[i]) == 0:
    val_predicted_tag.append([])
  else:
    for k in range(10):
      list_for_list.append(tag_embedding_model.wv.most_similar(positive = test_tag_list[i])[k][0])
    val_predicted_tag.append(list_for_list)
    print(i)
    list_for_list = []

In [23]:
test_tag_yu['predicted_tags']=val_predicted_tag
test_tag_yu

,index,tags,id,plylst_title,songs,like_cnt,updt_date,predicted_tags
0,1,"[나만의Best3, 인디아티스트들의추천음악]",7461,,"[196298, 269984, 267805, 175867, 529244, 63825...",0,2019-12-17 14:06:45.000,"[피쉬슈즈, 피터팬컴플렉스, 소수빈, 케이인디차트, 인디스트리트, 나만의best3,..."
1,2,[드라이브],90348,,"[273433, 331003, 68432, 411659, 117793, 616860...",21,2015-05-23 10:44:48.000,"[기분전환, 여행, 감각있는, 흥, 신나는, 여름곡, 윤식당시즌2, 신나는힙합, 빵..."
2,3,[분위기],58617,,"[702227, 48152, 440008, 358488, 701041, 540721...",0,2019-03-14 09:47:34.000,"[분위기있는, 관능적, 시크, 분위기깡패, 센치한, 홀로, 감성, wine, 라운지..."
3,5,[스트레스],88862,노래방 안에서,[],22,2016-01-22 20:55:12.000,"[기분전환, 내적댄스, 프로야구, 시원시원, 신나는노래모음, 국내edm, 박력, 고..."
4,7,[사랑],133975,봄바람과 듣고싶은 설레는 노래,[],0,2020-02-19 03:05:49.000,"[반함, 달달송, 진선미, 상사병, 썸, 남여, 첫만남, 이별, 이별후에_들으면_공..."
...,...,...,...,...,...,...,...,...
5417,10733,[별],77688,,"[93507, 324019, 221377, 656426, 26083, 661209,...",8,2018-08-10 20:14:19.000,"[편지, 밤하늘, 야경, 무중력, 푸른밤종현입니다, 저녁에듣기좋은노래, 몽롱, mo..."
5418,10735,[추억],137930,,"[323755, 397594, 445908, 570242, 221853, 20018...",16,2016-04-18 11:02:09.000,"[회상, 슈가맨시즌2, 올드감성, 토토가, 80년대생, 나가수, 그리움, 담시, 이..."
5419,10736,"[띵곡의, 우울, 분위기, 드라이브, 산책]",936,,"[105140, 582252, 199262, 422915, 547967, 48791...",1,2020-04-08 07:15:59.000,"[산책해요, 띵_곡, 우울하고, 오후의, 우울할, 생각을, 조용함, 아쿠스틱, 위로..."
5420,10737,[기분전환],110589,,"[21976, 207746, 40025, 31635, 567462, 641799, ...",6,2016-06-29 00:57:21.000,"[드라이브, 신나는, 스트레스, 여행, 핫한노래, 팝, 제주여행, 매장음악카페, 시..."


In [24]:
a = test_tag_yu['id']
b = test_tag_yu['predicted_tags']
c = pd.concat([a, b], axis=1)
c.to_json('H_result_1.json') #json 파일로 저장

In [25]:
tag2vec = c
tag2vec

,id,predicted_tags
0,7461,"[피쉬슈즈, 피터팬컴플렉스, 소수빈, 케이인디차트, 인디스트리트, 나만의best3,..."
1,90348,"[기분전환, 여행, 감각있는, 흥, 신나는, 여름곡, 윤식당시즌2, 신나는힙합, 빵..."
2,58617,"[분위기있는, 관능적, 시크, 분위기깡패, 센치한, 홀로, 감성, wine, 라운지..."
3,88862,"[기분전환, 내적댄스, 프로야구, 시원시원, 신나는노래모음, 국내edm, 박력, 고..."
4,133975,"[반함, 달달송, 진선미, 상사병, 썸, 남여, 첫만남, 이별, 이별후에_들으면_공..."
...,...,...
5417,77688,"[편지, 밤하늘, 야경, 무중력, 푸른밤종현입니다, 저녁에듣기좋은노래, 몽롱, mo..."
5418,137930,"[회상, 슈가맨시즌2, 올드감성, 토토가, 80년대생, 나가수, 그리움, 담시, 이..."
5419,936,"[산책해요, 띵_곡, 우울하고, 오후의, 우울할, 생각을, 조용함, 아쿠스틱, 위로..."
5420,110589,"[드라이브, 신나는, 스트레스, 여행, 핫한노래, 팝, 제주여행, 매장음악카페, 시..."


# 4. tag2vec에서 비어있는 데이터 처리하기

In [26]:
no_tag2vec_index = []
for i in range(len(tag2vec['predicted_tags'])):
  if len(tag2vec['predicted_tags'][i]) == 0:
    no_tag2vec_index.append(tag2vec['id'][i])

In [36]:
problemset2 = pd.DataFrame(no_tag2vec_index)
problemset2.columns = ['id']

In [43]:
mf_tag

,index,id,predicted_tags
0,1,7461,"[카페, 팝, 신나는, 뉴에이지, 피아노, 재즈, 댄스, 연주곡, 락, 스트레스]"
1,2,90348,"[겨울, 락, 크리스마스, 재즈, 캐롤, 눈, 연말, 매장음악, 따뜻한, 겨울감성]"
2,3,58617,"[사랑, 설렘, 봄, 이별, 감성, 겨울, 달달, 연애, 고백, 슬픔]"
3,5,88862,"[사랑, 설렘, 봄, 이별, 달달, 겨울, 연애, 고백, 연인, 감성]"
4,7,133975,"[사랑, 설렘, 봄, 까페, 달달, 연애, 고백, 힐링, 연인, 썸]"
...,...,...,...
5417,10733,77688,"[힙합, 랩, 알앤비, 소울, 신나는, 드라이브, 트렌디, HipHop, 그루브, 운동]"
5418,10735,137930,"[사랑, 이별, 슬픔, 설렘, 봄, 매장음악, 까페, 그리움, 힐링, 비오는날]"
5419,10736,936,"[신나는, 드라이브, 기분전환, 뉴에이지, 스트레스, 알앤비, 댄스, 카페, 운동,..."
5420,10737,110589,"[기분전환, 스트레스, 취향저격, 뉴에이지, 트렌디, 아침, 봄, 재즈, 출근길, 위로]"


In [61]:
mf_tag = pd.read_json('mf_tag.json')
problemset2_id=problemset2['id']

problemset2_tag=[]
for i in range(problemset2_id.shape[0]):
  B=mf_tag[problemset2_id[i]==mf_tag['id']].iloc[0,2]
  problemset2_tag.insert(i, B)

problemset2['predicted_tags']=problemset2_tag
problemset2_result=problemset2[['id', 'predicted_tags']]
problemset2_result.to_json('J_result_2.json')

In [62]:
problemset2_result

,id,predicted_tags
0,103795,"[이별, 슬픔, 비오는날, 눈물, 그리움, 스트레스, 우울, 매장음악, 헤어짐, 사랑]"
1,9823,"[힙합, 랩, 알앤비, 소울, HipHop, 그루브, 트렌디, RnB, 국내힙합, ..."
2,31103,"[새벽, 밤, 비오는날, 감성, 저녁, 분위기, 몽환, 여름, 새벽감성, 우울]"
3,75837,"[새벽, 밤, 비오는날, 감성, 저녁, 분위기, 몽환, 여름, 새벽감성, 우울]"
4,81933,"[휴식, 드라이브, 힐링, 스트레스, 운동, Pop, 저녁, 주말, 분위기, 트렌디]"
...,...,...
239,30510,"[힐링, 휴식, 뉴에이지, 피아노, 연주곡, 클래식, 자장가, 스트레스, 집중, 재즈]"
240,15803,"[신나는, 스트레스, 댄스, 기분전환, 인디, 운동, 뉴에이지, EDM, 클럽, 힐링]"
241,115035,"[잔잔한, 사랑, 이별, 새벽, 슬픔, 밤, 설렘, 비오는날, 겨울, 가을]"
242,77836,"[알앤비, 소울, RnB, 그루브, 분위기, 힙합, 트렌디, 재즈, 감각적인, soul]"


# 5. 다른 모델에서 부족한 데이터 채우기

Train2vec -> Tag2vec -> MF model 과정의 일부분. (자세한 내용은 read.me 4-1을 참고)

In [ ]:
tag_addition = pd.read_json('곡만 존재하는 플레이리스트에서 predicted_tags 출력.json')
tag_addition

,id,ply_token,tags,songs,predicted_songs,predicted_tags
0,70107,[],[],"[398985, 449403, 411543, 528044, 143048, 98020...",[],"[황현, 프리스타일, Apink, 고등학생]"
1,102395,[],[],"[528550, 285114, 506667, 339507, 653012, 435029]",[],"[움프, 하드스타일, futurehouse, MartinGarrix, RunDeep..."
2,42586,[],[],"[538867, 513110, 225168, 211527, 425074, 21709...",[],"[weeflow, kxan]"
3,13244,[],[],"[32042, 98472, 475222, 56796, 662086, 666787, ...",[],"[힙발라드, 에픽하이, 드렁큰타이거, 재지팩트, 화지, Beenzino, 허클베리피..."
4,55399,[],[],"[264320, 605720]",[],"[JPOP모음, 중국노래, 중국음악, 일본애니, JAPAN, 일본밴드, chanso..."
...,...,...,...,...,...,...
4293,120081,[],[],"[224607, 672096, 174950, 202906, 562039, 37070...",[],"[잔잔한락, 추억의락, ACDC, 틴스피릿, Grunge, Metallica, NW..."
4294,16489,[],[],"[260730, 243178, 19411, 311420, 55791, 406782,...",[],"[Kenzie, 폐막식, 켄지, 첸백시, 스엠, 엑소엘, 에프엑스, SM엔터테인먼트..."
4295,46470,[],[],"[67655, 65672, 210647, 608386, 413837, 566468,...",[],"[생각이, 생각을, 아픈, 자기전이나, 감정이입, 슬퍼서, 우울함은, 감성발라드, ..."
4296,48788,[],[],"[145292, 402779, 452316, 324334, 217189, 42301...",[],"[가요리믹스, 김경호, 지아]"


In [ ]:
not_tag_list_ida = [list(filter(lambda x: x in list(tag_embedding_model.wv.vocab.keys()), tag_addition['predicted_tags'][i])) for i in range(len(tag_addition['predicted_tags']))]

In [ ]:
list_for_list = []
test_predicted_tag = []
for i in range(len(not_tag_list_ida)):
  if len(not_tag_list_ida[i]) == 0:
    test_predicted_tag.append([])
  else:
    for k in range(10):
      list_for_list.append(tag_embedding_model.wv.most_similar(positive=not_tag_list_ida[i])[k][0])
    test_predicted_tag.append(list_for_list)
    print(i)
    list_for_list = []

In [ ]:
tag_addition['additional_tag']= test_predicted_tag
tag_addition

,id,ply_token,tags,songs,predicted_songs,predicted_tags,additional_tag
0,70107,[],[],"[398985, 449403, 411543, 528044, 143048, 98020...",[],"[황현, 프리스타일, Apink, 고등학생]","[서지음, 오늘의추천가수, 슴스테, Monotree, 잼팩토리, 모노트리, eone..."
1,102395,[],[],"[528550, 285114, 506667, 339507, 653012, 435029]",[],"[움프, 하드스타일, futurehouse, MartinGarrix, RunDeep...","[UMFKOREA2017, 바운드, 울트라뮤직페스티벌, ElectroHouse, 매..."
2,42586,[],[],"[538867, 513110, 225168, 211527, 425074, 21709...",[],"[weeflow, kxan]","[Empire, 히팝, 제국, 센힙합, SMTM, nb2, 홍대클럽홍대엔비, 일리네..."
3,13244,[],[],"[32042, 98472, 475222, 56796, 662086, 666787, ...",[],"[힙발라드, 에픽하이, 드렁큰타이거, 재지팩트, 화지, Beenzino, 허클베리피...","[VINXEN, 딥플로우, 사이먼도미닉, 키비, SIMONDOMINIC, SMTM,..."
4,55399,[],[],"[264320, 605720]",[],"[JPOP모음, 중국노래, 중국음악, 일본애니, JAPAN, 일본밴드, chanso...","[만화영화, 애니ost, 중음, 짱구, TV만화주제가, 그시절우리가좋아했던소녀, 쿵..."
...,...,...,...,...,...,...,...
4293,120081,[],[],"[224607, 672096, 174950, 202906, 562039, 37070...",[],"[잔잔한락, 추억의락, ACDC, 틴스피릿, Grunge, Metallica, NW...","[본조비, 고스트스테이션, MMFD, ji, mymadfatdiary, ChuckB..."
4294,16489,[],[],"[260730, 243178, 19411, 311420, 55791, 406782,...",[],"[Kenzie, 폐막식, 켄지, 첸백시, 스엠, 엑소엘, 에프엑스, SM엔터테인먼트...","[슴스테, 잼팩토리, 서지음, 방구석1열, EXO_CBX, 재현, 엔시티의나잇나잇,..."
4295,46470,[],[],"[67655, 65672, 210647, 608386, 413837, 566468,...",[],"[생각이, 생각을, 아픈, 자기전이나, 감정이입, 슬퍼서, 우울함은, 감성발라드, ...","[빈자리, 후폭풍, 가슴아픈노래, 이별후유증, 아프다, 리수, 우울할, 돋보이는, ..."
4296,48788,[],[],"[145292, 402779, 452316, 324334, 217189, 42301...",[],"[가요리믹스, 김경호, 지아]","[플라이투더스카이, MCTHEMAX, 노래방인기곡, 이별후에_듣는노래, 김종국, 슬..."


In [ ]:
for i in range(len(tag_addition['predicted_tags'])):
  a = len(tag_addition['predicted_tags'][i])
  if a != 10:
    b = tag_addition['additional_tag'][i][0:10-a]
    tag_addition['predicted_tags'][i] = tag_addition['predicted_tags'][i] + b

In [ ]:
a = tag_addition['id']
b = tag_addition['predicted_tags']
c = pd.concat([a,b], axis=1)
c.to_json('train2vec_addition.json')

In [ ]:
c

,id,predicted_tags
0,70107,"[황현, 프리스타일, Apink, 고등학생, 서지음, 오늘의추천가수, 슴스테, Mo..."
1,102395,"[움프, 하드스타일, futurehouse, MartinGarrix, RunDeep..."
2,42586,"[weeflow, kxan, Empire, 히팝, 제국, 센힙합, SMTM, nb2..."
3,13244,"[힙발라드, 에픽하이, 드렁큰타이거, 재지팩트, 화지, Beenzino, 허클베리피..."
4,55399,"[JPOP모음, 중국노래, 중국음악, 일본애니, JAPAN, 일본밴드, chanso..."
...,...,...
4293,120081,"[잔잔한락, 추억의락, ACDC, 틴스피릿, Grunge, Metallica, NW..."
4294,16489,"[Kenzie, 폐막식, 켄지, 첸백시, 스엠, 엑소엘, 에프엑스, SM엔터테인먼트..."
4295,46470,"[생각이, 생각을, 아픈, 자기전이나, 감정이입, 슬퍼서, 우울함은, 감성발라드, ..."
4296,48788,"[가요리믹스, 김경호, 지아, 플라이투더스카이, MCTHEMAX, 노래방인기곡, 이..."
